### Problem generator
Problem generator as given by professor, with default N values.

In [3]:
N_VALUES = [5, 10, 20, 100, 500, 1000]
SEED = 42

import logging
import random
from copy import copy
import random
import platform
from collections import Counter
from gx_utils import *

def problem(N, seed=None):
    '''Function to generate a set covering problem with number N, default seed is None'''
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

### Greedy solution
Greed solution as given by professor.

In [4]:
import logging

def greedy(N):
    '''Just adds lists sorted from shortest to longest until the set is covered'''
    goal = set(range(N))
    covered = set()
    solution = list()
    nodes = 0
    all_lists = sorted(problem(N, seed=SEED), key=lambda l: len(l))
    while goal != covered:
        x = all_lists.pop(0)
        nodes+=1
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
        f"Greedy solution for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%, Nodes visited: {nodes})"
    )
    logging.debug(f"{solution}")

In [5]:
logging.getLogger().setLevel(logging.INFO)
for N in N_VALUES:
    greedy(N)

INFO:root:Greedy solution for N=5: w=5 (bloat=0%, Nodes visited: 13)
INFO:root:Greedy solution for N=10: w=13 (bloat=30%, Nodes visited: 14)
INFO:root:Greedy solution for N=20: w=46 (bloat=130%, Nodes visited: 14)
INFO:root:Greedy solution for N=100: w=332 (bloat=232%, Nodes visited: 23)
INFO:root:Greedy solution for N=500: w=2162 (bloat=332%, Nodes visited: 28)
INFO:root:Greedy solution for N=1000: w=4652 (bloat=365%, Nodes visited: 27)


### "longest first" Greedy
Greedy solution as seen in the professor version, just starting from the longest random list first.
Performs generally worse, fairly better in just one case, N = 20. It is probably a random consequence of the list generation.

In [6]:
def longest_greedy(N):
    '''As the greedy, but the first list to be added is the longest'''
    goal = set(range(N))
    covered = set()
    solution = list()
    nodes=0
    all_lists = sorted(problem(N, seed=SEED), key=lambda l: len(l))
    first_iter = True
    while goal != covered:
        if first_iter:
            x = x = all_lists.pop()
            nodes+=1
            first_iter = False
        else:
            x = all_lists.pop(0)
            nodes+=1
        if not set(x) < covered:
            solution.append(x)
            covered |= set(x)

    logging.info(
            f"Alternate greedy version for N={N}: w={sum(len(_) for _ in solution)} (bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%, Nodes visited: {nodes})")


logging.getLogger().setLevel(logging.INFO)
for N in N_VALUES:
   longest_greedy(N)

INFO:root:Alternate greedy version for N=5: w=5 (bloat=0%, Nodes visited: 5)
INFO:root:Alternate greedy version for N=10: w=14 (bloat=40%, Nodes visited: 15)
INFO:root:Alternate greedy version for N=20: w=36 (bloat=80%, Nodes visited: 15)
INFO:root:Alternate greedy version for N=100: w=340 (bloat=240%, Nodes visited: 19)
INFO:root:Alternate greedy version for N=500: w=2187 (bloat=337%, Nodes visited: 28)
INFO:root:Alternate greedy version for N=1000: w=4699 (bloat=370%, Nodes visited: 28)


### Different solution
Another possible solution with the implementation of Dijkstra search.
Note - this is incomplete.

In [7]:
from gx_utils import *

class Graph:
    '''graph inizialization, compute'''
    def __init__(self, num_vert):
        self.v = num_vert
        self.edges = [[-1 for i in range(num_vert)] for j in range(num_vert)]
        self.visited = []

    def add_edge(self, u, v, weight):
        self.edges[u][v] = weight
        #self.edges[v][u] = weight

def create_graph(graph, problem_list):
    for i in range(graph.v):
        for j in range(graph.v):
            graph.add_edge(i, j, len(problem_list[j]))
            graph.add_edge(j, i, len(problem_list[i]))

In [8]:
def dijkstra_sol(N):

    GOAL = set(range(N))

    def problem_start(N):
        'starts the problem, adds the first vertex, so the shortest list, to the visited'
        p = sorted(problem(N, seed=SEED), key=lambda l: len(l))
        g = Graph(len(p))
        create_graph(g, p)

        return g, p

    def visited_to_set(state, problem_list):
        sol = [tuple(problem_list[e]) for e in state]
        return set(sum((e for e in sol), start=()))
    
    def goal_test(state, problem_list):
        return visited_to_set(state, problem_list) == GOAL
    
    _, p = problem_start(N)
   
    D = {v:float('inf') for v in range(_.v)}
    solution = p
    
    for start in range(_.v):  
        g, p = problem_start(N)
        pq = PriorityQueue()  
        pq.push((0, start), p=0)
        #pq.put((0, start))
        D = {v:float('inf') for v in range(_.v)}
        D[start]=0

        while pq:
            (dist, current_vertex) = pq.pop()
            g.visited.append(current_vertex)

            for neighbor in range(g.v):

                if g.edges[current_vertex][neighbor] != -1:
                    new_elements = visited_to_set(g.visited, p) - set(p[neighbor])
                    distance = g.edges[current_vertex][neighbor] - len(new_elements)

                    
                    if neighbor not in g.visited:
                        old_cost = D[neighbor]
                        cost = D[current_vertex] + distance

                        if cost < old_cost:
                            pq.push((cost, neighbor), p=cost)
                            D[neighbor] = cost
            
                    if goal_test(g.visited, p):

                        new_solution = [p[elem] for elem in g.visited]
                        weight_new = sum(len(_) for _ in new_solution)
                        weight_old = sum(len(_) for _ in solution)
                        solution = new_solution if weight_new<weight_old else solution
                        
                        if weight_new==N:
                            return solution
                        
                        pq = PriorityQueue()
        
    return solution

# incomplete and possibly non functioning approach, left it here to leave trace of what was delivered
# will probably complete/review it for learning purposes

In [15]:
logging.getLogger().setLevel(logging.DEBUG)

for N in [5, 10, 20]:
    solution = dijkstra_sol(N)
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%), solution is {solution}"
    )

INFO:root: Solution for N=5: w=5 (bloat=0%), solution is [[0], [1], [4], [2], [3]]
INFO:root: Solution for N=10: w=10 (bloat=0%), solution is [[8, 9, 3], [6], [0, 4], [2, 5], [1, 7]]
INFO:root: Solution for N=20: w=33 (bloat=65%), solution is [[6, 9, 11, 12, 17], [8, 4, 7], [18, 2, 15], [8, 16, 5], [1, 3, 13, 14], [17, 18, 7], [17, 10, 1, 7], [16, 9, 19, 6], [0, 1, 2, 7]]


NOTE: everything here is copied from professor Squillero's solution and is here for the sake of completion of the exercise and to keep track of the different algorithms.
It should not be considered as mine.

# Dijkstra's
Professor Squillero's solution, with added comments for learning purposes.

    Copyright © 2022 Giovanni Squillero <squillero@polito.it>
    https://github.com/squillero/computational-intelligence 
    Free for personal or classroom use; see 'LICENSE.md' for details.

In [10]:
from gx_utils import *

def dijkstra(N, all_lists):
    """Vanilla Dijkstra's algorithm"""
    
    # Goal and utility variables initialization 
    # all_lists is the problem generated sorted list 
    GOAL = set(range(N))

    # generates a tuple consisting of the lists of the problem
    # ordered and unchangeable list
    all_lists = tuple(set(tuple(_) for _ in all_lists))
    frontier = PriorityQueue()
    nodes = 0

    # converts a state (list of visited nodes) into a set
    # to be able to compare with goal
    def state_to_set(state):
        return set(sum((e for e in state), start=()))

    # comparison with goal, returns bool
    def goal_test(state):
        return state_to_set(state) == GOAL

    # returns a list of lists, which
    # could be the possible step -> all except 
    # subsets or equivalent sets
    def possible_steps(state):
        current = state_to_set(state)
        return [l for l in all_lists if not set(l) <= current]

    # computation of the internal weight
    # per state
    def w(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return sum(cnt[c] - 1 for c in cnt if cnt[c] > 1), -sum(cnt[c] == 1 for c in cnt)
    
    # keeps on updating the frontier based on 
    # internal weight computed by w() function
    # and then choosing the next state as the one with 
    # the lowest internal weight
    state = tuple()
    while state is not None and not goal_test(state):
        nodes += 1
        for s in possible_steps(state):
            frontier.push((*state, s), p=w((*state, s)))
        state = frontier.pop()

    logging.debug(f"dijkstra: SOLVED! nodes={nodes:,}; w={sum(len(_) for _ in state):,}; iw={w(state)})")
    return state

In [14]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20]:
    solution = dijkstra(N, problem(N, seed=42))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

INFO:root: Solution for N=5: w=5 (bloat=0%)
INFO:root: Solution for N=10: w=10 (bloat=0%)
INFO:root: Solution for N=20: w=23 (bloat=15%)


# Hill Climbing
Professor Squillero Hill Climbing solution, with added comments for learning purposes.

    Copyright © 2022 Giovanni Squillero <squillero@polito.it>
    https://github.com/squillero/computational-intelligence 
    Free for personal or classroom use; see 'LICENSE.md' for details.

In [20]:
def hc(N, all_lists):
    """Vanilla Hill Climber"""

    # same operation as for Dijkstra
    all_lists = set(tuple(_) for _ in all_lists)

    # state evaluation by number of lists in the counter
    # and total sum of the elements in them
    def evaluate(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    # tweak function on the current solution
    # randomly eliminates a list from the solution 
    # or adds a list non present in the solution 
    # from the possible lists
    def tweak(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while random.random() < 0.7:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    # sets a max number of consecutive useless steps
    # to stop tweaking
    current_solution = set()
    useless_steps = 0
    while useless_steps < 10_000:
        useless_steps += 1
        candidate_solution = tweak(current_solution)
        if evaluate(candidate_solution) > evaluate(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {evaluate(current_solution)}")
    return current_solution

In [28]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100, 1000]:
    solution = hc(N, problem(N, seed=42))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)" 
    )

INFO:root: Solution for N=5: w=5 (bloat=0%)
INFO:root: Solution for N=10: w=11 (bloat=10%)
INFO:root: Solution for N=20: w=24 (bloat=20%)
INFO:root: Solution for N=100: w=214 (bloat=114%)
INFO:root: Solution for N=1,000: w=3,383 (bloat=238%)


In [16]:
def hc_adapt(N, all_lists):
    """Vanilla Hill Climber"""

    # same operation as for Dijkstra
    all_lists = set(tuple(_) for _ in all_lists)

    # state evaluation by number of lists in the counter
    # and total sum of the elements in them
    def evaluate(state):
        cnt = Counter()
        cnt.update(sum((e for e in state), start=()))
        return len(cnt), -cnt.total()

    # tweak function on the current solution
    # randomly eliminates a list from the solution 
    # or adds a list non present in the solution 
    # from the possible lists
    def tweak0(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.7:
            r = random.choice(list(new_solution))
            new_solution.remove(r)
        while random.random() < 0.7:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution
    
    def tweak1(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.8:
            r = random.choice(sorted(list(new_solution), key=lambda x: len(x))[0:5])
            new_solution.remove(r)
        while random.random() < 0.8:
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution
    
    def tweak2(solution):
        new_solution = set(solution)
        while new_solution and random.random() < 0.8:
            r = sorted(list(new_solution), key=lambda x: len(x))[0]
            new_solution.remove(r)
            a = random.choice(list(all_lists - solution))
            new_solution.add(a)
        return new_solution

    # sets a max number of consecutive useless steps
    # to stop tweaking
    current_solution = set()
    useless_steps = 0
    while useless_steps < 10_000:
        useless_steps += 1

        if useless_steps < 5000:
            candidate_solution = tweak0(current_solution)
        elif 5000 < useless_steps < 8000:
            candidate_solution = tweak1(current_solution)
        else:
            candidate_solution = tweak2(current_solution)

        if evaluate(candidate_solution) > evaluate(current_solution):
            useless_steps = 0
            current_solution = copy(candidate_solution)
            logging.debug(f"New solution: {evaluate(current_solution)}")
    return current_solution

In [29]:
logging.getLogger().setLevel(logging.INFO)

for N in [5, 10, 20, 100, 500, 1000]:
    solution = hc_adapt(N, problem(N, seed=42))
    logging.info(
        f" Solution for N={N:,}: "
        + f"w={sum(len(_) for _ in solution):,} "
        + f"(bloat={(sum(len(_) for _ in solution)-N)/N*100:.0f}%)"
    )

INFO:root: Solution for N=5: w=5 (bloat=0%)
INFO:root: Solution for N=10: w=10 (bloat=0%)
INFO:root: Solution for N=20: w=24 (bloat=20%)
INFO:root: Solution for N=100: w=214 (bloat=114%)
INFO:root: Solution for N=500: w=1,509 (bloat=202%)
INFO:root: Solution for N=1,000: w=3,469 (bloat=247%)
